In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import itertools

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
df = pd.read_csv('./heart.csv')
df

In [ ]:
df.isnull().sum()

In [ ]:
df[df.duplicated()]

In [ ]:
df.drop_duplicates(keep='first', inplace=True)
df[df.duplicated()]

In [ ]:
df.reset_index(drop=True)
df

In [ ]:
df.describe()

In [ ]:
df_corr_mat = df.corr()
px.imshow(df_corr_mat)


In [ ]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [ ]:
X

In [ ]:
list(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
print(y_train.shape)
print(y_test.shape)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
all_models = []


In [ ]:
def optimize(models: list,
            X_train: np.array,
            y_train: np.array,
            X_test: np.array,
            y_test: np.array,
            epochs: int = 50,
            verbose: int = 0) -> pd.DataFrame:
  
  # We'll store the results here
  results = []
  
  def train(model: tf.keras.Sequential) -> dict:
    # Change this however you want 
    # We're not optimizing this part today
    model.compile(
      loss=tf.keras.losses.binary_crossentropy,
      optimizer=tf.keras.optimizers.Adam(),
      metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy')
      ]
    )
    
    # Train the model
    model.fit(
      X_train,
      y_train,
      epochs=epochs,
      verbose=verbose
    )
    
    # Make predictions on the test set
    preds = model.predict(X_test)
    prediction_classes = [1 if prob > 0.5 else 0 for prob in np.ravel(preds)]
    
    # Return evaluation metrics on the test set
    return {
      'model_name': model.name,
      'test_accuracy': accuracy_score(y_test, prediction_classes),
      'test_precision': precision_score(y_test, prediction_classes),
      'test_recall': recall_score(y_test, prediction_classes),
      'test_f1': f1_score(y_test, prediction_classes)
    }
  
  # Train every model and save results
  index = 0
  for model in models:
    try:
      print(model.name, end=' ... ')
      res = train(model=model)
      results.append(res)
      model.save(os.path.join('.', 'models', f'{index}_{model.name}', ''))
    except Exception as e:
      print(f'{model.name} --> {str(e)}')
    index += 1
      
  return pd.DataFrame(results)

In [ ]:
optimization_results = optimize(
    models=all_models,
    X_train=X_train_scaled,
    y_train=y_train,
    X_test=X_test_scaled,
    y_test=y_test
)

In [ ]:
test = tf.keras.models.load_model(os.path.join('.', 'models', '0dense16_dense16_dense16', ''))
test_pred = test.predict(X_test)
test_prediction_classes = [1 if prob > 0.5 else 0 for prob in np.ravel(test_pred)]

curr_pred = all_models[0].predict(X_test)
curr_prediction_classes = [1 if prob > 0.5 else 0 for prob in np.ravel(curr_pred)]
print(precision_score(y_test, test_prediction_classes))
print(precision_score(y_test, curr_prediction_classes))

In [ ]:
type(optimization_results)

In [ ]:
optimization_results.sort_values(by='test_precision', ascending=False)

In [ ]:
optimization_results_precision = optimization_results.sort_values(by='test_precision', ascending=False)
optimization_results_precision = optimization_results_precision.reset_index()
optimization_results_precision

In [ ]:
optimization_results_recall = optimization_results.sort_values(by='test_recall', ascending=False)
optimization_results_recall = optimization_results_recall.reset_index()
optimization_results_recall

In [ ]:
optimization_results_accuracy = optimization_results.sort_values(by='test_accuracy', ascending=False)
optimization_results_accuracy = optimization_results_accuracy.reset_index(drop=True)
optimization_results_accuracy

In [ ]:
plt.plot(optimization_results['test_precision'][:])
plt.show()

In [ ]:
plt.plot(optimization_results_precision['test_precision'])
plt.show()

print(optimization_results_precision['test_precision'].shape)

In [ ]:
plt.plot(optimization_results_recall['test_recall'])
plt.show()